# Part 1

In [1]:
example = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [2]:
import pandas as pd

In [3]:
example_df = pd.DataFrame([list(x) for x in example.split('\n')])
example_df

,0,1,2,3,4,5,6,7,8,9
0,4,6,7,.,.,1,1,4,.,.
1,.,.,.,*,.,.,.,.,.,.
2,.,.,3,5,.,.,6,3,3,.
3,.,.,.,.,.,.,#,.,.,.
4,6,1,7,*,.,.,.,.,.,.
5,.,.,.,.,.,+,.,5,8,.
6,.,.,5,9,2,.,.,.,.,.
7,.,.,.,.,.,.,7,5,5,.
8,.,.,.,$,.,*,.,.,.,.
9,.,6,6,4,.,5,9,8,.,.


In [4]:
# find all the numbers first.

import string
DIGITS = set(string.digits)

# create a dict of positions (i, j) to numbers

def compute_pos_to_num_dict(df):
    pos_to_num_dict = {}
    for i, row in df.iterrows():
        for j, elt in enumerate(row):
            if elt in DIGITS:
                # if the previous digit is a number, continue - we shouldn't do num substrings
                if j > 0 and row[j-1] in DIGITS:
                    continue
                # figure out the whole number. This is the start of the number
                num_str = elt
                for k in range(j+1, len(row)):
                    if row[k] in DIGITS:  # the number continues
                        num_str += row[k]                    
                    else:  # the number has ended
                        pos_to_num_dict[(i, j)] = int(num_str)
                        num_str = ''
                        break
                # if num_str isn't empty, then it was at the end of the row
                if num_str:
                    pos_to_num_dict[(i, j)] = int(num_str)

    return pos_to_num_dict

pos_to_num_dict = compute_pos_to_num_dict(example_df)
pos_to_num_dict

{(0, 0): 467,
 (0, 5): 114,
 (2, 2): 35,
 (2, 6): 633,
 (4, 0): 617,
 (5, 7): 58,
 (6, 2): 592,
 (7, 6): 755,
 (9, 1): 664,
 (9, 5): 598}

In [5]:
# for each number and its starting position, get a list of all other surrounding positions
    
def compute_surrounding_pos_to_check(pos, num, df):
    row = pos[0]
    col = pos[1]

    existing_pos = set()
    for j in range(0, len(str(num))):
        existing_pos.add((row, col+j))

    pos_to_check = set()
    min_row = row-1
    max_row = row+1
    min_col = col-1
    max_col = col+len(str(num))

    for i in range(min_row, max_row+1):
        for j in range(min_col, max_col+1):
            if i >= 0 and i < df.shape[0] and j >= 0 and j < df.shape[1] and (i, j) not in existing_pos:
                pos_to_check.add((i, j))

    return pos_to_check

SET_TO_CHECK = {'#', '$', '%', '&', '*', '+', '-', '/', '=', '@'}

def is_part_number(pos, num, df):
    pos_to_check = compute_surrounding_pos_to_check(pos, num, df)
    any_symbols = False
    for (i, j) in pos_to_check:
        if df.iloc[i, j] != '.':
            return True
    return False
        
pos = (2, 6)
num = 633
is_part_number(pos, num, example_df)

True

In [6]:
pos_to_num_dict = compute_pos_to_num_dict(example_df)

total = 0
for pos, num in pos_to_num_dict.items():
    if is_part_number(pos, num, example_df):
        total += num
total

4361

In [7]:
real_input = open('day_3.txt').read().split('\n')[:-1]
df = pd.DataFrame([list(x) for x in real_input])
df.shape

(140, 140)

In [8]:
pos_to_num_dict = compute_pos_to_num_dict(df)

total = 0
for pos, num in pos_to_num_dict.items():
    if is_part_number(pos, num, df):
        total += num
total

546563

# Part 2

In [25]:
from collections import defaultdict

# Return the location of the attached gear, if it exists, for this particular position and number
def get_gear(pos, num, df):
    pos_to_check = compute_surrounding_pos_to_check(pos, num, df)
    any_symbols = False
    for (i, j) in pos_to_check:
        if df.iloc[i, j] == '*':
            return (i, j)
    return None

def get_total_gear_ratio(df):
    pos_to_num_dict = compute_pos_to_num_dict(df)

    # Build a map: gear position to a list of numbers attached to it
    gear_dict = defaultdict(list)
    for pos, num in pos_to_num_dict.items():
        gear_pos = get_gear(pos, num, df)
        if gear_pos is not None:
            gear_dict[gear_pos].append(num)

    total_gear_ratio = 0
    for pos, nums in gear_dict.items():
        if len(nums) == 2:
            gear_ratio = nums[0] * nums[1]
            total_gear_ratio += gear_ratio

    return total_gear_ratio

get_total_gear_ratio(example_df)

467835

In [26]:
get_total_gear_ratio(df)

91031374